# Install Dependencies

In [1]:
!pip install sklearn
!pip install matplotlib
!pip install scrapy

!pip install fastText
!pip install spacy
!pip install nltk
!pip install tensorflow
!pip install tensorflow_hub

  Running setup.py bdist_wheel for sklearn ... done
  Stored in directory: /Users/biksingh/Library/Caches/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn
    100% |████████████████████████████████| 256kB 5.0MB/s ta 0:00:01
  Running setup.py bdist_wheel for PyDispatcher ... done
  Stored in directory: /Users/biksingh/Library/Caches/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
Successfully built PyDispatcher


# Import Dependencies

In [2]:
import numpy as np
import pickle
import csv
import json
import re
from pprint import pprint

from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

import nltk
from nltk.corpus import stopwords
import fastText
import spacy
import tensorflow as tf
import tensorflow_hub as hub

import logging
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.selector import Selector
from urllib.parse import urljoin

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# Settings for notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Show Python version
import platform
platform.python_version()

'3.6.5'

# Crawling Data

## Get Offenders Info

### Setup a pipeline

In [4]:
class OffenderInfoWriterPipeline(object):
    def open_spider(self, spider):
        self.file = open('data/offender_info_results.json', 'w+')
    def close_spider(self, spider):
        self.file.close()
    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

### Define the spider

In [5]:
class OffenderInfoSpider(scrapy.Spider):
    name = "OffenderInfo"
    start_urls = [
        'http://www.tdcj.state.tx.us/death_row/dr_executed_offenders.html'
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.OffenderInfoWriterPipeline': 1}
    }

    def parse(self, response):
        print('A response from %s just arrived!', response.url)
        sel = Selector(response)

        table = sel.xpath('//table[@class="tdcj_table indent"]/tr')
        for tr in table[1:]:
            url_info = urljoin(response.url, str(tr.xpath('td[2]/a/@href').extract_first()))
            url_stmt = urljoin(response.url, str(tr.xpath('td[3]/a/@href').extract_first()))
        
            yield {
                'first_name': tr.xpath('td[5]/text()').extract_first(),
                'last_name': tr.xpath('td[4]/text()').extract_first(),
                'age': tr.xpath('td[7]/text()').extract_first(),
                'date': tr.xpath('td[8]/text()').extract_first(),
                'race': tr.xpath('td[9]/text()').extract_first(),
                'country': tr.xpath('td[10]/text()').extract_first(),
                'info_link': url_info,
                'death_note_link': url_stmt
            }
        

### Start the crawler

In [8]:
process = CrawlerProcess()
process.crawl(OffenderInfoSpider)
process.start()

2018-11-25 18:03:29 [scrapy.utils.log] INFO: Scrapy 1.5.1 started (bot: scrapybot)
2018-11-25 18:03:29 [scrapy.utils.log] INFO: Versions: lxml 4.2.5.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.19.0, Twisted 17.5.0, Python 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) - [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 18.0.0 (OpenSSL 1.0.2p  14 Aug 2018), cryptography 2.3.1, Platform Darwin-17.7.0-x86_64-i386-64bit
2018-11-25 18:03:29 [scrapy.crawler] INFO: Overridden settings: {'LOG_LEVEL': 30}


In [9]:
file = open('data/offender_info_results.json', 'r')
lines = file.readlines()
line = lines[0]
obj = json.loads(line)
pprint([key for key in obj])

FileNotFoundError: [Errno 2] No such file or directory: 'data/offender_info_results.json'

## Extract Death Note

In [3]:
urls = []
objs = []

file = open('data/offender_info_results.json', 'r')
lines = file.readlines()
for line in lines:
    objs.append(json.loads(line))
    urls.append(objs[-1]['death_note_link'])

### Setup a pipeline

In [4]:
class DeathNoteWriterPipeline(object):
    def open_spider(self, spider):
        self.file = open('data/death_note_results.json', 'w+')
    def close_spider(self, spider):
        self.file.close()
    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item
   

### Define the spider

In [5]:
import logging
from scrapy.selector import Selector

class DeathNoteSpider(scrapy.Spider):
    name = "DeathNote"
    start_urls = urls
    
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.DeathNoteWriterPipeline': 1}
    }

    def parse(self, response):     
        sel = Selector(response)
    
        first = str(sel.xpath('//div[@id="content_right"]/p[6]/text()').extract_first()).strip()
        second = str(sel.xpath('//div[@id="content_right"]/p[7]/text()').extract_first()).strip()
        
        death_note = ''
        if first and first != 'Last Statement:' and first != 'None':
            death_note += first
        if second and second != 'None':
            death_note += second
            
        url = response.url
        obj = [o for o in objs if o['death_note_link'] == url][0]
        obj['death_note'] = death_note
    
        yield obj

### Start the crawler

In [6]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})
process.crawl(DeathNoteSpider())
process.start()

2018-11-25 18:05:33 [scrapy.utils.log] INFO: Scrapy 1.5.1 started (bot: scrapybot)
2018-11-25 18:05:33 [scrapy.utils.log] INFO: Versions: lxml 4.2.5.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.19.0, Twisted 17.5.0, Python 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) - [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 18.0.0 (OpenSSL 1.0.2p  14 Aug 2018), cryptography 2.3.1, Platform Darwin-17.7.0-x86_64-i386-64bit
2018-11-25 18:05:33 [scrapy.crawler] INFO: Overridden settings: {'LOG_LEVEL': 30, 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


# Data Cleaning

In [7]:
file = open('data/death_note_results.json', 'r')
lines = file.readlines()
objs = []
for line in lines:
    objs.append(json.loads(line))

## String Format Converting

In [8]:
for i in range(len(objs)):
    note = objs[i]['death_note']
    if len(note.strip()) > 0:
        note = (note.encode('ascii', 'ignore')).decode("utf-8")
        objs[i]['death_note'] = note

## Remove Punctuations and Stopwords

In [9]:
stop_words = set(stopwords.words('english')) - set('not')
rm_punc = re.compile('[^a-zA-Z]')

In [12]:
objs_clean = []
death_notes_clean = []
for i in range(len(objs)):
    note = objs[i]['death_note']
    note = rm_punc.sub(' ', note)
    words = []
    tokens = note.split()
    for token in tokens:
        if token and token not in stop_words and token.lower() not in stop_words \
        and token.lower() != 'none':
            words.append(token)
    
    if words:
        note = ' '.join(words)
        death_notes_clean.append(note)
        objs[i]['death_note'] = note
        objs_clean.append(objs[i])

In [13]:
with open('data/death_note_clean.pickle', 'wb') as handle:
    pickle.dump(objs_clean, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
len(death_notes_clean)

449

## Lemmatization

In [15]:
nlp = spacy.load('en_core_web_lg')

In [16]:
def clean(text):
    doc = nlp(text)
    tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
    return ' '.join(tokens)

In [17]:
objs_lemma = []
death_notes_lemma = []
for i in range(len(objs_clean)):
    obj = objs_clean[i]
    
    note = clean(obj['death_note'])
    death_notes_lemma.append(note)
    obj['death_note'] = note
    objs_lemma.append(obj)

In [18]:
with open('data/death_notes_lemma.pickle', 'wb') as handle:
    pickle.dump(objs_lemma, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Bag-of-Words

In [ ]:
#per doc
bag_of_words = {}
statement_word_count = {}
for s_id in range(len(last_words)):
    words_list = last_words.iloc[s_id]['LastStatement'].split()
    statement_word_count[s_id] = len(words_list)
    bag_of_words[s_id] = {}
    for word in words_list:
        if word in bag_of_words[s_id]:
            bag_of_words[s_id][word] = bag_of_words[s_id][word] +1
        else:
            bag_of_words[s_id][word] = 1

for s_id in bag_of_words:
    sorted_x = dict(sorted(bag_of_words[s_id].items(), key=lambda kv: kv[1], reverse=True))
    bag_of_words[s_id] = sorted_x

In [ ]:
# for all doc
statementFreqDict = {}

for s_id in bag_of_words:
    for word in bag_of_words[s_id]:
        if word in statementFreqDict:
            statementFreqDict[word] = statementFreqDict[word] + bag_of_words[s_id][word]
        else:
            statementFreqDict[word] = bag_of_words[s_id][word]
        
sorted_x = dict(sorted(statementFreqDict.items(), key=lambda kv: kv[1], reverse=True))
statementFreqDict = sorted_x


## Claculating TFIDF

In [ ]:
def tf(ngram, doc_id, ngram_count, ngramDocDict):
    return ngramDocDict[doc_id][ngram] / ngram_count[doc_id]

def n_containing(ngram, ngramDocDict):
    return sum(1 for doc_id in ngramDocDict if ngram in ngramDocDict[doc_id])

def idf(ngram, ngramDocDict):
    return math.log(len(ngramDocDict) / (n_containing(ngram, ngramDocDict)))

def tfidf(ngram, doc_id, ngram_count, ngramDocDict):
    return tf(ngram, doc_id, ngram_count, ngramDocDict) * idf(ngram, ngramDocDict)

In [ ]:
def cal_tfidf_doc(ngramDocDict):
    
    ngram_statement_tfidf = {}
    for s_id in ngramDocDict:
        ngram_statement_tfidf[s_id] = {}

        for ngram in ngramDocDict[s_id]:
            ngram_statement_tfidf[s_id][ngram] = tfidf(ngram, s_id, statement_word_count, ngramDocDict)
            
    return ngram_statement_tfidf

In [ ]:
statement_tfidf = cal_tfidf_doc(bag_of_words)

## Printing top 10 TFIDF

In [ ]:
from heapq import heappush, heappop

heap = []

for s_id in statement_tfidf:
    for word in statement_tfidf[s_id]:
        heappush(heap, (statement_tfidf[s_id][word], word))
        
        if len(heap) > 10:
            heappop(heap)

In [ ]:
while(heap):
    r = heappop(heap)
    print(r[1], r[0])

# Word Embedding

In [ ]:
death_notes = death_notes_lemma
# death_notes = death_notes_clean

## Elmo

In [ ]:
elmo_url = "https://tfhub.dev/google/elmo/2"
elmo_model = hub.Module(elmo_url)

In [ ]:
# Reduce logging output.
# tf.logging.set_verbosity(tf.logging.ERROR)

# with tf.Session() as session:
#     session.run([tf.global_variables_initializer(), tf.tables_initializer()])
#     elmo_embedding = session.run(elmo_model(doc_txt))

In [ ]:
embed_file = "data/elmo_embedding.pickle"
with open(embed_file, 'wb') as handle:
    pickle.dump(elmo_embedding, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Clustering

In [ ]:
embedding = glove_embedding
# embedding = spacy_embedding
# embedding = ft_embedding
# embedding = use_embedding
# embedding = elmo_embedding

In [ ]:
print(np.array(embedding).shape)

## Elbow Method

In [ ]:
from sklearn import metrics
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import numpy as np

maximum = 25

_ = plt.plot()
distortions = []
K = range(2, maximum+1)
for k in K:
    kmeans = KMeans(n_clusters=k, init='k-means++', random_state=2018)
    _ = kmeans.fit(embedding)
    _ = distortions.append(kmeans.inertia_)

# Plot the elbow
_ = plt.plot(K, distortions, 'bx-')
_ = plt.xlabel('k')
_ = plt.ylabel('Distortion')
_ = plt.title('The Elbow Method showing the optimal k')
plt.show()

## K-Mean

In [ ]:
k = 5
kmeans = KMeans(n_clusters=k, init='k-means++', random_state=2018).fit(embedding)
clusters = kmeans.predict(embedding)

## Get notes in each clsuter

In [ ]:
note_clusters = {}
for c in set(clusters):
    note_clusters[c] = []
    for i, txt in enumerate(death_notes):
        if clusters[i] == c:
            note_clusters[c].append(txt)

# Results Analysis

## Clustering Visualization

In [ ]:
arr = np.array(embedding)
tsne = TSNE(n_components=2, random_state=512)
reduced = tsne.fit_transform(arr)       
t = np.array(reduced).transpose()

In [ ]:
from matplotlib.lines import Line2D

markers = ["x", "v", "o", "s", "*", ">", "<", "P", 
           '1', '2', '3', '4', 'h', "d", "|", "+"]
colors = ['darkorange', 'steelblue', 'limegreen',  'salmon', 'y',  'violet', 'c', 'tomato', 
          'rosybrown', 'brown', 'darkmagenta', 'pink', 'gold', "orange", "skyblue", "seagreen"]

fig, ax = plt.subplots(figsize=(10, 10))

for x, y, c in zip(t[0], t[1], clusters):
    _ = ax.scatter(x, y, c=colors[c], marker=markers[c])

types = []
for c in set(clusters):
    types.append(Line2D([], [], color=colors[c], marker=markers[c], label=c))

# for i, c in enumerate(clusters):
#     if c == 5:
#         _ = ax.annotate(doc_txt[i], (t[0][i], t[1][i]), fontsize=18)

_ = plt.legend(handles=types, loc='lower left')
plt.show()

In [ ]:
for i in range(len(clusters)):
    if clusters[i] == 3:
        print(death_notes_ori[i])
        print()

# Top K Words